# 첫번째 시도 - 실패

In [ ]:
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3
from scipy import linalg
import numpy as np

# 평균과 공분산 계산 함수
def calculate_activation_statistics(x, model, device):
    # Inception 모델의 중간층(layer)까지 모델 정의
    model.eval()
    model.to(device)
    upsample = torch.nn.Upsample(size=(299, 299), mode='bilinear').to(device)
    block_idx = inception_v3.BLOCK_INDEX_BY_DIM[2048]
    model = torch.nn.Sequential(
        *list(model.children())[:block_idx+1]
    )

    # 데이터셋의 데이터를 Inception 모델에 입력하여 중간층의 출력값을 추출
    n = x.shape[0]
    x = upsample(x)
    with torch.no_grad():
        x = model(x.to(device))
        x = F.adaptive_avg_pool2d(x, (1, 1)).squeeze(-1).squeeze(-1)

    # 평균과 공분산 계산
    mean = torch.mean(x, dim=0)
    cov = torch.matmul(x.T, x) / n - torch.matmul(mean.T, mean)
    return mean.cpu().numpy(), cov.cpu().numpy()

# FID 계산 함수
def calculate_fid(real_images, fake_images, model, device):
    # 실제 데이터셋에 대한 평균과 공분산 계산
    m1, s1 = calculate_activation_statistics(real_images, model, device)

    # 생성된 데이터셋에 대한 평균과 공분산 계산
    m2, s2 = calculate_activation_statistics(fake_images, model, device)

    # 평균 제곱근 차이와 공분산 합계의 제곱근 계산
    diff = m1 - m2
    covmean, _ = linalg.sqrtm(s1.dot(s2), disp=False)
    if not np.isfinite(covmean).all():
        offset = np.eye(s1.shape[0]) * 1e-6
        covmean = linalg.sqrtm((s1 + offset).dot(s2 + offset))
    fid = np.dot(diff, diff) + np.trace(s1 + s2 - 2 * covmean)
    return fid

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inception V3 모델 불러오기
model = inception_v3(pretrained=True, transform_input=False, aux_logits=True)

In [ ]:
# real_images와 fake_images를 텐서 형태로 준비
# 이 텐서들은 적절한 전처리를 거친 후에 생성되어야 함
real_images = torch.randn(32, 3, 299, 299)  # 예시를 위해 무작위 텐서 사용
fake_images = torch.randn(32, 3, 299, 299)  # 예시를 위해 무작위 텐서 사용

# FID 값을 계산
fid = calculate_fid(real_images, fake_images, model, device)
print("FID score:", fid)

# 두번째 시도 - 실패

In [ ]:
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3
from scipy import linalg
import numpy as np

# 평균과 공분산 계산 함수
def calculate_activation_statistics(x, model, device):
    model.eval()
    model.to(device)
    upsample = torch.nn.Upsample(size=(299, 299), mode='bilinear').to(device)

    # 데이터셋의 데이터를 Inception 모델에 입력하여 중간층의 출력값을 추출
    n = x.shape[0]
    x = upsample(x)
    with torch.no_grad():
        x = model(x.to(device))  # 주목: 중간층의 출력값만 사용
        x = F.adaptive_avg_pool2d(x, (1, 1)).squeeze(-1).squeeze(-1)

    # 평균과 공분산 계산
    mean = torch.mean(x, dim=0)
    cov = torch.matmul(x.T, x) / n - torch.matmul(mean.T, mean)
    return mean.cpu().numpy(), cov.cpu().numpy()


# FID 계산 함수
def calculate_fid(real_images, fake_images, model, device):
    # 실제 데이터셋에 대한 평균과 공분산 계산
    m1, s1 = calculate_activation_statistics(real_images, model, device)

    # 생성된 데이터셋에 대한 평균과 공분산 계산
    m2, s2 = calculate_activation_statistics(fake_images, model, device)

    # 평균 제곱근 차이와 공분산 합계의 제곱근 계산
    diff = m1 - m2
    covmean, _ = linalg.sqrtm(s1.dot(s2), disp=False)
    if not np.isfinite(covmean).all():
        offset = np.eye(s1.shape[0]) * 1e-6
        covmean = linalg.sqrtm((s1 + offset).dot(s2 + offset))
    fid = np.dot(diff, diff) + np.trace(s1 + s2 - 2 * covmean)
    return fid


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inception V3 모델 불러오기
model = inception_v3(pretrained=True, transform_input=False, aux_logits=True)

# Inception 모델의 중간층(layer)까지 모델 정의
# 여기에서, inception_v3에서 미리 정의된 중간층 인덱스를 사용할 수 없으므로,
# 미리 정의된 인덱스 값인 9를 직접 사용합니다.
model = torch.nn.Sequential(
    *list(model.children())[:9+1]
)

In [ ]:
# real_images와 fake_images를 텐서 형태로 준비
# 이 텐서들은 적절한 전처리를 거친 후에 생성되어야 함
real_images = torch.randn(32, 3, 299, 299)  # 예시를 위해 무작위 텐서 사용
fake_images = torch.randn(32, 3, 299, 299)  # 예시를 위해 무작위 텐서 사용

# FID 값을 계산
fid = calculate_fid(real_images, fake_images, model, device)
print("FID score:", fid)

In [ ]:
fake_images.max()

In [ ]:
real_images.shape

In [ ]:
import torch
from PIL import Image
from torchvision import transforms

# 입력 이미지의 크기가 256x256인 경우
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
])

path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/IXI0_fake_B.png"
# 이미지 불러오기
image = Image.open(path).convert('RGB')
# 이미지를 tensor로 변환하기
fake_images = torch.tensor(np.array(image)).permute(2, 0, 1).unsqueeze(0).float()
# 출력
print(fake_images.shape)

path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/IXI0_real_B.png"
# 이미지 불러오기
image = Image.open(path).convert('RGB')
# 이미지를 tensor로 변환하기
real_images = torch.tensor(np.array(image)).permute(2, 0, 1).unsqueeze(0).float()
# 출력
print(real_images.shape)

# FID 값을 계산
fid = calculate_fid(real_images, fake_images, model, device)
print("FID score:", fid)

In [ ]:
from torchvision import transforms

# 입력 이미지의 크기가 256x256인 경우
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
])
image = Image.open(path).convert('RGB')
fake_images = transform(image).unsqueeze(0).to(device)

In [ ]:
from torchvision import transforms

# 입력 이미지의 크기가 256x256인 경우
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
])

path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/IXI0_real_B.png"

image = Image.open(path).convert('RGB')
fake_images = transform(image).unsqueeze(0).to(device)

In [ ]:
print(fake_images.shape)

# 세번째 시도 - 실패
출처: https://github.com/mseitzer/pytorch-fid

In [1]:
pip install pytorch-fid

Note: you may need to restart the kernel to use updated packages.


In [28]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch

from pytorch_fid import fid_score

# 실제 이미지 폴더 경로
real_images_path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/real"

# 생성된 이미지 폴더 경로
fake_images_path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/fake"

# 이미지 데이터셋 생성
real_images_dataset = dset.ImageFolder(root=real_images_path,
                                       transform=transforms.Compose([
                                           transforms.Resize(256),
                                           transforms.CenterCrop(256),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                                std=[0.5, 0.5, 0.5])
                                       ]))
fake_images_dataset = dset.ImageFolder(root=fake_images_path,
                                       transform=transforms.Compose([
                                           transforms.Resize(256),
                                           transforms.CenterCrop(256),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                                std=[0.5, 0.5, 0.5])
                                       ]))

# 데이터로더 생성
real_images_dataloader = torch.utils.data.DataLoader(real_images_dataset,
                                                      batch_size=32,
                                                      shuffle=True,
                                                      num_workers=8)
fake_images_dataloader = torch.utils.data.DataLoader(fake_images_dataset,
                                                      batch_size=32,
                                                      shuffle=True,
                                                      num_workers=8)

# FID 계산
fid_value = fid_score.calculate_fid_given_paths([real_images_dataloader, fake_images_dataloader], dims=2048, batch_size=32, device='cuda')
print('FID score:', fid_value)

FileNotFoundError: Couldn't find any class folder in /SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/real.

In [30]:
from torchvision.datasets import DatasetFolder

# 실제 이미지 폴더 경로
real_images_path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/real"

# 생성된 이미지 폴더 경로
fake_images_path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/fake"

# 이미지 데이터셋 생성
real_images_dataset = DatasetFolder(root=real_images_path,
                                    loader=lambda x: Image.open(x),
                                    extensions=".png",
                                    transform=transforms.Compose([
                                        transforms.Resize(256),
                                        transforms.CenterCrop(256),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                             std=[0.5, 0.5, 0.5])
                                    ]))
fake_images_dataset = DatasetFolder(root=fake_images_path,
                                    loader=lambda x: Image.open(x),
                                    extensions=".png",
                                    transform=transforms.Compose([
                                        transforms.Resize(256),
                                        transforms.CenterCrop(256),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                             std=[0.5, 0.5, 0.5])
                                    ]))

# 데이터로더 생성
real_images_dataloader = torch.utils.data.DataLoader(real_images_dataset,
                                                      batch_size=32,
                                                      shuffle=True,
                                                      num_workers=8)
fake_images_dataloader = torch.utils.data.DataLoader(fake_images_dataset,
                                                      batch_size=32,
                                                      shuffle=True,
                                                      num_workers=8)

# FID 계산
fid_value = fid_score.calculate_fid_given_paths([real_images_dataloader, fake_images_dataloader], dims=2048, batch_size=32, device='cuda')
print('FID score:', fid_value)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not DataLoader

In [33]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch

from pytorch_fid import fid_score
import os

# 실제 이미지 폴더 경로
real_images_path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/real"

# 생성된 이미지 폴더 경로
fake_images_path = "/SSD3_8TB/Daniel/06_pGAN/pGAN-cGAN/results/1_pGAN_run_align/test_latest/images/fake"

# 이미지 데이터셋 생성
real_images_dataset = dset.ImageFolder(root=real_images_path,
                                       transform=transforms.Compose([
                                           transforms.Resize(256),
                                           transforms.CenterCrop(256),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                                std=[0.5, 0.5, 0.5])
                                       ]))
fake_images_dataset = dset.ImageFolder(root=fake_images_path,
                                       transform=transforms.Compose([
                                           transforms.Resize(256),
                                           transforms.CenterCrop(256),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                                std=[0.5, 0.5, 0.5])
                                       ]))

# 데이터로더 생성
real_images_dataloader = torch.utils.data.DataLoader(real_images_dataset,
                                                      batch_size=32,
                                                      shuffle=True,
                                                      num_workers=8)
fake_images_dataloader = torch.utils.data.DataLoader(fake_images_dataset,
                                                      batch_size=32,
                                                      shuffle=True,
                                                      num_workers=8)

# FID 계산
paths = [real_images_path, fake_images_path]
# fid_value = fid_score.calculate_fid_given_paths(paths, dims=2048, batch_size=32, device='cuda')
# fid_value = fid_score.calculate_fid_given_paths(paths, dims=2048, batch_size=None, device='cuda')
fid_value = fid_score.calculate_fid_given_paths(paths, dims=2048, batch_size=32, device='cuda')



print('FID score:', fid_value)

ValueError: batch_size should be a positive integer value, but got batch_size=0